In [ ]:
from googlesearch import search

# to search
query = '"AI trends" (site:theverge.com OR site:techcrunch.com OR site:cio.com OR site:forbes.com OR site:venturebeat.com OR site:digitaltrends.com OR site:itprotoday.com OR site:computerworld.com OR site:zdnet.com OR site:bloomberg.com OR site:businessinsider.com OR site:edtechmagazine.com OR site:edsurge.com OR site:psfk.com OR site:springwise.com OR site:technode.com OR site:techeu.com OR site:wundermanthompson.com OR site:36kr.com) after:2024-06-01 before:2024-06-12'

urls = []
for url in search(query, tld="co.in", num=10, stop=10, pause=2):
    urls.append(url)

In [ ]:
urls = '''https://www.forbes.com/sites/adrianbridgwater/2024/06/10/pega-opens-mind-to-left--right-brain-ai-development/
https://www.forbes.com/sites/committeeof200/2024/06/04/ai-insights-for-executives-business-lessons-from-the-magnificent-7/
https://www.itprotoday.com/ai-machine-learning/itpro-today-s-top-10-stories-about-ai-in-2022
https://www.forbes.com/sites/lanceeliot/2024/06/04/the-nitty-gritty-about-that-latest-risk-of-ai-letter-and-a-vaunted-call-for-a-right-to-forewarn/
https://www.businessinsider.com/retro-anti-instagram-app-built-by-ex-meta-staffers-2024-5
https://www.itprotoday.com/regulatory-compliance/california-lawmaker-unveils-landmark-ai-bill
https://www.forbes.com/sites/johnbuckingham/
https://www.businessinsider.com/prime
https://www.businessinsider.com/category/social-media
https://www.businessinsider.com/category/instagram'''.split("\n")

In [ ]:
from langchain_community.document_loaders import AsyncChromiumLoader, AsyncHtmlLoader
from langchain_community.document_transformers import BeautifulSoupTransformer, Html2TextTransformer